# Imports

In [6]:
# code libraries
import numpy as np
import matplotlib.pyplot as plt
from vector_class import TripleVector
import random

# code from simple single source
# from simple_singlePoint_source import visualize


# Parameters

In [7]:
A_min = 1e3 # Bq
A_max = 2e3 # Bq
A_b = 5e-5 # Bq
h = 10 # m
dt = 100 # the pause on each point od the grid in s
x_max = 4; sigma_x = 0.1 # m
y_max = 4; sigma_y = 0.1 # m
grid = 8
n_bins = 20
K = 0.1 # is somewhere in the interval [0, 1]
F = 0.140 # factor for inhilation of Pu-239 in mSV/Bq

radiation = {"A_min": A_min, "A_max": A_max, "A_b": A_b, "dt": dt, "dose_factor": F}
detector = {"h": h, "x_max": x_max, "y_max": y_max, "grid": grid, "detector_constant": K} # the detector constant tells us the quality of the 
                                                                                          # detector



## Flyover

In [17]:
NORTH, S, W, E = (0, -1), (0, 1), (-1, 0), (1, 0) # directions
turn_left = {NORTH: W, E: NORTH, S: E, W: S} # old -> new direction

def spiral(width, height):
    if width < 1 or height < 1:
        raise ValueError
    x, y = width // 2, height // 2 # start near the center
    dx, dy = NORTH # initial direction
    matrix = [[None] * width for _ in range(height)]
    count = 0
    while True:
        count += 1
        matrix[y][x] = count # visit
        # try to turn right
        new_dx, new_dy = turn_left[dx,dy]
        new_x, new_y = x + new_dx, y + new_dy
        if (0 <= new_x < width and 0 <= new_y < height and
            matrix[new_y][new_x] is None): # can turn right
            x, y = new_x, new_y
            dx, dy = new_dx, new_dy
        else: # try to move straight
            x, y = x + dx, y + dy
            if not (0 <= x < width and 0 <= y < height):
                return matrix # nowhere to go

def print_matrix(matrix):
    width = len(str(max(el for row in matrix for el in row if el is not None)))
    fmt = "{:0%dd}" % width
    for row in matrix:
        print(" ".join("_"*width if el is None else fmt.format(el) for el in row))

spiral(5, 5)


[[25, 24, 23, 22, 21],
 [10, 9, 8, 7, 20],
 [11, 2, 1, 6, 19],
 [12, 3, 4, 5, 18],
 [13, 14, 15, 16, 17]]

In [13]:
# The goal is to improve the code so that the drone flies over the grid in a way that it firs locates the "hotspot" tile and then gathers the 
# information around it source. It dose this by flying around it in circles

def activity(source, x, y, h, ru=0, rv=0):
    u, v, A0 = source[0], source[1], source[2] # u, v are the coordinates of the source and A0 is its activity
    return (A0*(ru**2 + rv**2 + h**2)) / ((x - (u - ru))**2 + (y - (v - rv))**2 + h**2)

def point_source(x_max, y_max):
    return [random.uniform(-x_max, x_max),random.uniform(-y_max, y_max)]

def improv_flyOver(radiation, detector, source = []):
    A_min, A_max, A_b, dt = radiation["A_min"], radiation["A_max"], radiation["A_b"], radiation["dt"]
    h, x_max, y_max, grid, K = detector["h"], detector["x_max"], detector["y_max"], detector["grid"], detector["detector_constant"]
    N_grid = grid
    dx, dy = (2*x_max)/N_grid, (2*y_max)/N_grid

    
    # grid_x_noise, grid_y_noise = np.zeros((N_grid, N_grid)), np.zeros((N_grid, N_grid))
    xs = np.linspace(-x_max + dx/2, x_max - dx/2, int(N_grid))
    ys = np.flip(np.linspace(-y_max + dy/2, y_max - dy/2, int(N_grid)))
    grid_x, grid_y = np.meshgrid(xs, ys)

    data = [A_b, K, F, dt]
    map = np.zeros((N_grid, N_grid))

    if len(source) == 0:
        source = point_source(x_max, y_max)


    i, j = N_grid - 1, 0
    x = grid_x[i, j], y = grid_y[i, j]
    HD_max = max([dose_speed(source, x + dx, y, h, *data), dose_speed(source, x + dx, y + dy, h, *data), dose_speed(source, x, y + dy, h), *data])
    while dose_speed(source, x, y, h, *data) < HD_max:
        if (i == (N_grid - 1)) and (j == 0):
            print("Beginnig")
        elif j == 0:
            print("Left")
        elif j == (N_grid -1):
            print("Right")
        elif i == 0:
            print("Top")
        elif i == (N_grid - 1):
            print("Bottom")
        else:
            print("Middle")
    
    return

def dose_speed(source, x, y, h, A_b, K, F, dt):
    A = activity(source, x, y, h)
    A_det = A * (1 - K)
    N = np.random.poisson(A_det*dt)
    N_b = np.random.poisson(A_b*dt) # background radiation
    HD = F * (N + N_b)
    dHD = F * np.sqrt(N + N_b)
    return [HD, dHD]




